# Guided Project: Exploring ebay Car Sales Data


in this guided project, im gonna work with dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

you can find the dataset [here](https://data.world/data-society/used-cars-data).

the aim of thid project is to clean data and analyze the included used car listings. 

has made a few modifications from the original dataset:
* We sampled 50,000 data points from the full dataset, to ensure your code runs quickly in our hosted environment
* We dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

#### objetivo
o proposito desse projeto é por em pratica conceita que eu parendi sobre a limpeza e análise de dados.


### mais informacoes sobre o dataset 
o dicionario dos dados é disposto dos seguintes dados:

* `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
* `name` - Name of the car.
* `seller` - Whether the seller is private or a dealer.
* `offerType` - The type of listing
* `price` - The price on the ad to sell the car.
* `abtest` - Whether the listing is included in an A/B test.
* `vehicleType` - Tipo de Veículo.
* `yearOfRegistration` - The year in which the car was first registered.
* `gearbox` - The transmission type.
* `powerPS` - The power of the car in PS.
* `model` - The car model name.
* `kilometer` - How many kilometers the car has driven.
* `monthOfRegistration` - The month in which the car was first registered.
* `fuelType` - What type of fuel the car uses.
* `brand` - The brand of the car.
* `notRepairedDamage` - If the car has a damage which is not yet repaired.
* `dateCreated` - The date on which the eBay listing was created.
* `nrOfPictures` - The number of pictures in the ad.
* `postalCode` - The postal code for the location of the vehicle.
* `lastSeenOnline` - When the crawler saw this ad last online.

In [ ]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv", encoding="Latin-1")

In [ ]:
autos

In [ ]:
autos.info()
autos.head()

no total temos 50000 entradas e 20 colunas, podemos ver que 5 colunas estao sem valores.


# Data cleaning

## convertendo nome das colunas

iremos converter as colunas da forma camelcase para snake case.

tambem iremos renomear algumas colunas de um jeito mais descritivo.
* `Registration` to `registration_year`
* `monthOfRegistration` to `registration_month`
* `notRepairedDamage` to `unrepaired_damage`
* `dateCreated` to `ad_created`    

In [ ]:
print(autos.columns)

In [ ]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gear_box', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']
autos.head()

## renomeando palavras em alemão

iremos renomear os valores das colunas: `veichle_type`,`gear_box`,`fuel_type` e `unrepaired_damage`.

In [ ]:
autos["vehicle_type"].unique()

In [ ]:
autos["gear_box"].unique()

In [ ]:
autos["fuel_type"].unique()

In [ ]:
autos["unrepaired_damage"].unique()

In [ ]:
translate={"kleinwagen":"mini_car",
           "kombi":"estate car",
          "cabrio":"convertible",
          "andere":"other",
           'elektro' : "electric",
        'benzin' : 'petrol',
        'manuell' : "manual", 
        'automatik' : "automatic",
        "bus" : "bus",
        "limousine" : "limousine",
                 "coupe":"coupe",
                 "suv" : "suv",
                 "lpg" : "lpg",
                 "diesel" : "diesel",
                 "cng" : "cng",
                 "hybrid": "hybrid",
                 "Unknown" : "Unknown",
                 "nein" : "no",
                 "ja" : "yes",}

colunas = ["vehicle_type","gear_box","fuel_type","unrepaired_damage"]

In [ ]:
for x in colunas:
    autos[x] = autos[x].map(translate)
    

#replace nan values with "Unknown"
for c in colunas:
    autos.loc[autos[c].isnull(), autos[c]] =  "Unknown"

In [ ]:
autos["gear_box"].unique()

In [ ]:
autos

## convertendo datas

In [ ]:
autos

In [ ]:
autos.describe(include="all")

In [ ]:
autos.info()

## exploração inicial 

as colunas `seller` e `offer_type` possuem apenas dois valores quais apenas um de cada coluna se repetem apenas uma vez.

na coluna `registration_year` temos o valor 0 que nao condiz quando os meses vao de 1 a 12. 

algumas colunas estao como texto quando o ideal seria int ou float por exemplo: `price` e `odometer`

`nr_of_pictures` todos os valores são 0.

# dropping colunas

`nr_of_pictures`, `seller` e `offer_type` nao possuem relevancia e serao descartadas.


In [ ]:
autos = autos.drop(["nr_of_pictures","seller","offer_type"], axis=1)
autos.columns

# convertendo `price`, `odometer` para `int` e deletar caracteres não númericos

vamos deletar caracteres não númericos das colunas `price ` e `odometer`

depois o data type de cada um dessas colunas vai ser mudado para `int`

para melhor analize a coluna `odometer` vai ser renomeada como `odomer_km`

In [ ]:
prices = [] 

for each in autos["price"]:
    each = each.replace("$","")
    each = each.replace(",","")
    prices.append(each)

autos["price"] = prices
autos["price"] = autos["price"].astype(int)

autos["price"].head()

In [ ]:
odometer = []
for each in autos["odometer"]:
    each = each.replace("km","")
    each = each.replace(",","")
    odometer.append(each)
    
autos["odometer"] = odometer
autos["odometer"] = autos["odometer"].astype(int)

autos.rename({"odometer":"odometer_km"}, axis=1, inplace=True)

autos["odometer_km"].head()

# explorando `price`, `odometer_km` mais afundo

vamos checar em cada coluna se ha algum row de dado anómolo que pode ser droppado.

analizamos as culunas usando valores minimos e maximo e procurando por qualquer valor que parece irrealistico alto ou baixo(outliers) que possamos querer remover.




In [ ]:
autos["price"].describe().apply(lambda x: format(x, "f"))

In [ ]:
# price column

autos["price"].value_counts().sort_index(ascending=True).head(20)


In [ ]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

In [ ]:
autos["price"].value_counts(bins=4)

no topo dos valores da coluna `price` temos valores acima de 1 milhao, enquanto no final a diferença é muito grande com valores apartir de 0. um dos motivos de preços tao abaixo da media é que alguns desses carros sao para leiloes, e outro é que o preço indica uma negociacao para definir o preço final. 

para prevenir que esses valores nao inclinem os dados, vamos remover as entradas de preços abaixo de 100 e maiores que 350.000.

In [ ]:
autos
autos = autos[autos["price"].between(100,350001)]
autos

oque antes era 50000 entradas diminuimos em mais ou menos 3,55%, agora temos 48224 entradas. 

In [ ]:
# odometer_km
autos["odometer_km"].shape

In [ ]:
autos["odometer_km"].describe()

In [ ]:
autos["odometer_km"].value_counts().sort_index(ascending=True).head(15)

na coluna `odometer_km` como analizamos a cima os valores tem padroes, nao é qualquer valor especifico que pode ser inserido, isso é otimo pois nos da uma media melhor e como os valores em uma distribuicao ate que ok, não vejo nececissade de remover outliers nessa coluna. 

# explorando as colunas de data

5 colunas representam valores de datas, sendo elas: 
* `date_crawled`
* `last_seen`
* `ad_created`
* `registration_month`
* `registration_year`

no momento as colunas `date_crawled`, `last_seen`, e `ad_created` estao identificadas como strings no pandas. vamos transformar para uma representacao numerica para que possamos entender quantitativamente.

as outras duas colunas estao representadsas com valores numericos. 


In [ ]:
autos[["date_crawled","ad_created","last_seen"]][0:5]

os primeiros 10 caracteres representam o dia. para entender o range das datas, podemos extrair apenas os valores do dia, usando `Series.value_counts()` para gerar uma distribuicao, e ai ordenar pelo index.

In [ ]:
# date_crawled

print(autos["date_crawled"].str[:10].unique().shape)

date_crawled = pd.to_datetime(autos["date_crawled"].str[:10])

date_crawled.value_counts(normalize=True,dropna=False).sort_index(ascending=True)

In [ ]:
# ad_created
print(autos["ad_created"].str[:10].unique().shape)

ad_created = pd.to_datetime(autos["ad_created"].str[0:10])
ad_created.value_counts(normalize=True,dropna=False).sort_index(ascending=True)

a coluna `ad_created` tem datas de mais de um ano de diferença e quase o o dobro do numero de entradas em relacao a coluna `date_crawled`, essa coluna tem uma porcentagem muito pequenas nas maioras para as maiorias das datas.

In [ ]:
# last_seen
print(autos["last_seen"].str[:10].unique().shape)

last_seen = pd.to_datetime(autos["last_seen"].str[0:10])

autos["last_seen"] = last_seen

last_seen.value_counts(normalize=True,dropna=False).sort_index(ascending=True)



os dados da `last_seen` nos mostra que a uma boa parte dos dados estao concentrado nas ultimas 3 datas. `last_seen` se refere aos anuncios que foram retirados do site, seja por motivo de venda ou ter sido retirado. nao é como se nos ultimos 3 dias tivesse um aumento na venda, mas sim os crawlers apenas viram que carros foram removidos nessa data.  

# explorando dados do `registration_year`

In [ ]:
autos["registration_year"].describe()

o ano que o carro esta registrado geralmente indica em que ano ele foi produzido. o valor minimo é 1000, bem antes dos carros serem iventados, e o maximo é 9999 muito anos a frente. 

In [ ]:
autos["registration_year"].value_counts().sort_index(ascending=True).head(20)

 `registration_year` qualquer veiculo com o ano de fabricacao depois de 2016 é totalmente impreciso. determinar um ano de partida é um pouco mais incerto. 
 * vamos usar como parametro os carros com data de fabricacão entre 1950-2016.

In [ ]:
autos = autos[autos["registration_year"].between(1949,2017)]

In [ ]:
autos["registration_year"].value_counts(normalize=True, bins=5).sort_index(ascending=True)

a maioria dos carros tem o ano de fabricacao entre 1990-2002 com 48% ou 2003-2016 tambem com 48%. 



# explorando carros pela marca

vamos analizar a media de preços para as marcas com mais presença no data set

In [ ]:
autos["brand"].value_counts().index

In [ ]:
autos["brand"].value_counts(normalize=True)

vamos selecionar as marcas com no minimo 2,5% de carros de todo data set.

In [ ]:
brand = autos["brand"].value_counts(normalize=True)

select_brand = brand[brand > 0.050].index


price_dict = {}

for each in select_brand:
    brand_row = autos[autos["brand"]== each]
    mean = brand_row["price"].mean()
    price_dict[each] = int(mean)
    
    
####
    
print("\n")
sort_orders = sorted(price_dict.items(), key=lambda x: x[1], reverse=True)

for i in sort_orders:
    print(i[0], i[1])

## explorando quilometros rodados pelas marcas mais dominantes 

In [ ]:
mil_dict={}
    
for each in select_brand:
    miles = autos[autos["brand"]==each][["odometer_km"]].mean()
    mil_dict[each]=int(miles)
    
mil_dict

In [ ]:
miles_series = pd.Series(mil_dict)
price_series = pd.Series(price_dict)

price_series.describe()

In [ ]:
miles_series.describe()

In [ ]:
dataframe = pd.DataFrame(price_series,columns=["mean_prices"])
dataframe["mean_miles"] = miles_series

dataframe.sort_values(by=['mean_prices'], ascending=False)

autos.columns

para as 6 marcas de maior numero nesse data set temos: 

* a media de preços: 6383 e com uma variacao de 6303 entre os valores minimos e maximos. 

* a media de quilometros rodados: 129409.66 Km, e com pouca variacao entre os extremos.

a relacao entre preco e quilometros rodados sugerem valores inversamente propocionais, mas as marcas `ford` e  `opel` nostrazem dados questionaveis, para verificar melhor veremos a condicao do carro como mais um fator que deve impactar nos preços
